In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

strokedata = pd.read_csv('healthcare-dataset-stroke-data.csv')
strokedata

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [15]:
ols_object = sm.ols(formula = 'stroke~avg_glucose_level+age*bmi', data = strokedata)
model = ols_object.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 stroke   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     84.22
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           2.82e-69
Time:                        13:18:59   Log-Likelihood:                 1051.9
No. Observations:                4909   AIC:                            -2094.
Df Residuals:                    4904   BIC:                            -2061.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0189      0.022     -0.863      0.388      -0.062       0.024
avg_glucose_level     0.0004    6.6e-05      6.164      0.000       0.000       0.001
age                   0.0012      0.000      2.320      0.020       0.000       0.002
bmi                  -0.0026      0.001     -3.185      0.001      -0.004      -0.001
age:bmi            3.365e-05   1.85e-05      1.818      0.069   -2.63e-06    6.99e-05
==============================================================================
Omnibus:                     4033.628   Durbin-Watson:                   0.132
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            66703.790
Skew:                           4.082   Prob(JB):                         0.00
Kurtosis:                      19.108   Cond. No.                     1.19e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.19e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""